In [2]:
#Importing Packages
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

#Reading datasets
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [9]:
test

,user_id,book,rating
0,35859,The Divine Secrets of the Ya-Ya Sisterhood: A ...,7
1,147045,The Red Tent (Bestselling Backlist),8
2,85256,Fall On Your Knees (Oprah #45),8
3,276050,The Tale of the Body Thief (Vampire Chronicles...,5
4,127432,Four To Score (A Stephanie Plum Novel),10
5,101606,The Search,7
6,128732,A Is for Alibi (Kinsey Millhone Mysteries (Pap...,8
7,134900,Where the Heart Is (Oprah's Book Club (Paperba...,9
8,113270,Vinegar Hill (Oprah's Book Club (Paperback)),9
9,162562,She's Come Undone (Oprah's Book Club),9


In [3]:
#Question 2
#Part a

#Cosine similarity: Arguments(matrix, vector a, vector b)
def cos(mat, a, b):
    #Are the vectors same?
    if a == b: 
        return 1
    
    #Finding items that are rated by both users. 
    aval = mat.T[a].nonzero()
    bval = mat.T[b].nonzero()
    corated = np.intersect1d(aval, bval)
    
    #If no item is rated by both users, then similarity = 0
    if len(corated) == 0:
        return 0
    
    #Whats happening?
    avec = np.take(mat.T[a], corated)
    bvec = np.take(mat.T[b], corated)
    val = 1 - cosine(avec, bvec)
    
    #If the value is not a number, make similarity = 0
    if np.isnan(val):
        return 0
    
    return val
    
def pr(mat, a, b, imean):
    if a == b:
        return 1
    
    aval = mat.T[a].nonzero()
    bval = mat.T[b].nonzero()
    corated = np.intersect1d(aval, bval)
    
    if len(corated) < 2:
        return 0
    avec = np.take(mat.T[a], corated)
    bvec = np.take(mat.T[b], corated)
    avec1 = avec - imean[a]
    bvec1 = bvec - imean[b]
    val = 1 - cosine(avec1, bvec1)
    
    if np.isnan(val):
        return 0
    return val

def itemsimilar(mat, option):
    # *Calculate amean, umean and imean as before
    n = mat.shape[1]
    # *initialize a zero matrix with dimensions n,n to get the similarity matrix
    if option == 'pr':
        #print("PR")
        for i in range(n):
            for j in range(n):
                sim_mat[i][j] = pr(mat, i, j, imean)
        sim_mat = (sim_mat + 1)/2
    elif option == 'cos':
        #print("COS")
        for i in range(n):
            for j in range(n):
                sim_mat[i][j] = cos(mat, i, j)
    else:
        #print("Default")
        sim_mat = cosine_similarity(mat.T)
    return sim_mat, amean, umean, imean

In [10]:
#Part b
def predict(user, item, mat, item_similarity, amean, umean, imean, k=20):
    nzero = mat[user].nonzero()[0]
    if len(nzero) == 0:
        return amean
    baseline = imean + umean[user] - amean
    choice = nzero[item_similarity[item, nzero].argsort()[::-1][:k]]
    prediction = ((mat[user, choice] - baseline[choice]).dot(item_similarity[item, choice])/
                  sum(item_similarity[item, choice])) + baseline[item]
    if np.isnan(prediction):
        prediction = amean
    if prediction > 10:
        prediction = 10
    if prediction < 1:
        prediction = 1
    return prediction

In [ ]:
def get_results(train_data, test_data, option, rows, cols, k):
    # *initialize a zero matrix called full_mat with dim(rows,cols)
    for row in train_data:
        full_mat[row[0], row[1]] = row[2]
    item_similarity, amean, umean, imean = itemsimilar(full_mat, option)
    # *initialize empty preds and real lists.
    # *For each row in train_data, call the predict function using the above values as parameters,
    #  as explained before for the naïve prediction. Keep updating the preds and real lists.
    # *Calculate the RMSE error of both lists.
    print('Train Error')
    print('RMSE : %.4f' % err1)
    # *Reinitialize the preds and real lists to empty lists
    # *For each row in the test_data, call the predict function using the above values as parameters,
    #  as explained before for the naïve prediction. Keep updating the preds and real lists.
    # *Calculate the RMSE error of both lists.
    print('Test Error')
    print('RMSE : %.4f' % err2)